In [6]:
import numpy as np
import pandas as pd
# from icecream import ic
import subprocess
import json
import os
# import plotly.graph_objects as go
from matplotlib import pyplot as plt
# import geopandas as gpd
# from shapely.geometry import Polygon
# from shapely.ops import cascaded_union, unary_union
# from shapely import geometry
# from geopy import distance
%matplotlib inline
# import plotly.express as px 
from scipy import signal
from tqdm import tqdm
# from Bio import Phylo
# import ete3
import itertools
import sklearn

In [7]:
variants = pd.read_csv('variants_full.csv')


In [82]:
len(variants['gene'].unique())

58

In [83]:
a = ['dnaA','gyrB','gyrA','Rv0010c','fgd1','mshA','ccsA','Rv0565c','hadA','nusG','rpoB','rpoC','mmpL5','mmpS5','Rv0678','rplC','Rv1129c','fbiC','Rv1258c','embR','atpE','rrs','rrl','inhA','tsnR','bacA','ndh','katG','Rv1979c','eis','ahpC','Rv2477c','pepQ','Rv2752c','Rv2983','Rv3083','whiB7','lpqB','mtrB','mtrA','fbiA','fbiB','rpoA','ddn','glpK','embC','embA','embB','aftB','ubiA','ethA','ethR','whiB6']

In [16]:
cryptic = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data1/CRyPTIC_reuse_table_20221019_cleaned.csv')
overlap = set(variants['sample_id']).intersection(set(cryptic['wgs_id'].to_list()))
variants = variants[variants['sample_id'].isin(overlap)]
variants['SNP'] = variants['gene'] + '-'+ variants['change']

In [19]:
def compare_snp_lists(set_list, query_list):
    # Convert query_list to a set for O(1) lookup time
    query_set = query_list

    # Use numpy for efficient array operations
    import numpy as np
    # Convert set_list to a NumPy array for vectorized operations
    set_array = np.array(set_list)
    # Create an array of zeros with the same length as set_list
    output_array = np.zeros(len(set_list), dtype=int)
    # Update positions in output_array to 1 where elements of set_array are in query_set
    output_array[np.isin(set_array, query_set)] = 1

    return output_array.tolist()

# Example usage
set_list = ['SNP1', 'SNP2', 'SNP3', 'SNP4']
query_list = ['SNP2', 'SNP4']
output_list = compare_snp_lists(set_list, query_list)
print(output_list)  # Expected Output: [0, 1, 0, 1]


[0, 1, 0, 1]


In [20]:
def compare_snp_lists_with_values(set_list, query_list, values_list):
    # Create a dictionary from query_list and values_list for faster lookups
    query_dict = dict(zip(query_list, values_list))
    
    # Initialize output list with zeros
    output_list = [0] * len(set_list)
    
    # Iterate through the set list and update the output list
    for i, snp in enumerate(set_list):
        if snp in query_dict:
            # Use the corresponding value from values_list instead of 1
            output_list[i] = query_dict[snp]
    
    return output_list

# Example usage:
set_list = ['SNP1', 'SNP2', 'SNP3', 'SNP4']
query_list = ['SNP2', 'SNP4']
values_list = [5, 10]  # Corresponding values for 'SNP2' and 'SNP4'
output_list = compare_snp_lists_with_values(set_list, query_list, values_list)
print(output_list)  # Expected output: [0, 5, 0, 10]


def compare_snp_lists_with_values_optimized(set_list, query_list, values_list):
    # Create a dictionary from query_list and values_list for direct mapping
    query_dict = dict(zip(query_list, values_list))
    
    # Use list comprehension to build the output list directly
    output_list = [query_dict.get(snp, 0) for snp in set_list]
    
    return output_list

# Example usage
set_list = ['SNP1', 'SNP2', 'SNP3', 'SNP4']
query_list = ['SNP2', 'SNP4']
values_list = [5, 10]  # Corresponding values for 'SNP2' and 'SNP4'
output_list = compare_snp_lists_with_values_optimized(set_list, query_list, values_list)
print(output_list)  # Expected output: [0, 5, 0, 10]


[0, 5, 0, 10]
[0, 5, 0, 10]


In [33]:
variants.head()

,sample_id,genome_pos,gene,change,freq,type,sublin,drtype,drugs,SNP
514,ERR4829684,7362,gyrA,p.Glu21Gln,1.000000,missense_variant,lineage4.3.3,Sensitive,NaN,gyrA-p.Glu21Gln
515,ERR4829684,7585,gyrA,p.Ser95Thr,1.000000,missense_variant,lineage4.3.3,Sensitive,NaN,gyrA-p.Ser95Thr
516,ERR4829684,8040,gyrA,p.Gly247Ser,1.000000,missense_variant,lineage4.3.3,Sensitive,NaN,gyrA-p.Gly247Ser
517,ERR4829684,9304,gyrA,p.Gly668Asp,1.000000,missense_variant,lineage4.3.3,Sensitive,NaN,gyrA-p.Gly668Asp
518,ERR4829684,576108,mshA,p.Ala254Gly,0.218182,missense_variant,lineage4.3.3,Sensitive,NaN,mshA-p.Ala254Gly
...,...,...,...,...,...,...,...,...,...,...
7420745,ERR4797048,4267647,aftB,p.Asp397Gly,1.000000,missense_variant,lineage2.2.1,Pre-XDR-TB,NaN,aftB-p.Asp397Gly
7420746,ERR4797048,4338595,whiB6,c.-75delG,1.000000,upstream_gene_variant,lineage2.2.1,Pre-XDR-TB,NaN,whiB6-c.-75delG
7420747,ERR4797048,4407588,gid,c.615A>G,1.000000,synonymous_variant,lineage2.2.1,Pre-XDR-TB,NaN,gid-c.615A>G
7420748,ERR4797048,4407620,gid,p.Tyr195His,1.000000,missense_variant,lineage2.2.1,Pre-XDR-TB,NaN,gid-p.Tyr195His


In [ ]:
aa = []
all_snp = variants['SNP'].unique()
for x in tqdm(variants['sample_id'].unique()):
    aa.append(compare_snp_lists_with_values(all_snp, variants[variants['sample_id']==x]['SNP'].to_list(), variants[variants['sample_id']==x]['freq'].to_list()))

In [ ]:
variants[variants['sample_id']==x]['SNP'].to_list()

In [ ]:
aa_array = np.array(aa)
np.savetxt('data_aa/aa_data.csv', aa_array, delimiter=',')   # X is an array

In [23]:
aa_array = np.loadtxt('data_aa/aa_data.csv', delimiter=',')   # X is an array

In [24]:
cryptic_mic = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC/data1/CRyPTIC_reuse_table_20221019_cleaned.csv')

In [37]:
variants['sample_id'].unique()

array(['ERR4829684', 'ERR8975196', 'ERR3287611', ..., 'ERR4811968',
       'ERR2510245', 'ERR4797048'], dtype=object)

In [67]:
aa_array

array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 0., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [1., 1., 0., ..., 0., 0., 0.],
       [1., 1., 0., ..., 0., 0., 1.],
       [1., 1., 0., ..., 0., 0., 0.]])

In [ ]:
cryptic_mic['wgs_id'].

In [49]:
len(cryptic_mic.columns)

43

In [51]:
cryptic_mic[cryptic_mic['wgs_id'].isin(variants['sample_id'].unique())].iloc[:,42]

0        ERR2184202
1        ERR2184203
2        ERR2184204
3        ERR2184205
4        ERR2184206
            ...    
12121    ERR8976118
12122    ERR8976119
12123    ERR8976120
12125    ERR8976122
12126    ERR8976123
Name: wgs_id, Length: 11920, dtype: object

# Getting all snp data

In [52]:
mic_aa = cryptic_mic[cryptic_mic['wgs_id'].isin(variants['sample_id'].unique())]#.iloc[:,14:27]

In [58]:
#ordering mic like in aa_array
mic_aa['wgs_id'] = pd.Categorical(mic_aa['wgs_id'], categories=variants['sample_id'].unique().tolist(), ordered=True)
mic_aa = mic_aa.sort_values('wgs_id')

/tmp/ipykernel_106569/2574372831.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mic_aa['wgs_id'] = pd.Categorical(mic_aa['wgs_id'], categories=variants['sample_id'].unique().tolist(), ordered=True)


In [75]:
aa_array.shape

(11920, 27063)

In [76]:
aa_array[:int(aa_array.shape[0]*0.9), :].shape

(10728, 27063)

In [77]:
np.savetxt('data_aa/aa_data_train.csv', aa_array[:int(aa_array.shape[0]*0.9), :], delimiter=',', fmt='%s')
# Save the remaining 10% of rows to another CSV file
np.savetxt('data_aa/aa_data_test.csv', aa_array[int(aa_array.shape[0]*0.9):, :], delimiter=',', fmt='%s')

In [63]:

mic_aa = mic_aa.iloc[:,14:27].astype('float')
mic_aa.to_csv('data_aa/mic_aa.csv', index=False)

In [70]:
mic_aa.iloc[:int(mic_aa.shape[0]*0.9),:].to_csv('data_aa/mic_aa_train.csv', index=False)
mic_aa.iloc[int(mic_aa.shape[0]*0.9):,:].to_csv('data_aa/mic_aa_test.csv', index=False)

In [56]:
variants['sample_id'].unique()

array(['ERR4829684', 'ERR8975196', 'ERR3287611', ..., 'ERR4811968',
       'ERR2510245', 'ERR4797048'], dtype=object)

In [59]:
mic_aa.iloc[:,42]

8157     ERR4829684
11361    ERR8975196
1244     ERR3287611
7641     ERR4828904
9905     ERR4831761
            ...    
5125     ERR4812175
2539     ERR4797355
4936     ERR4811968
464      ERR2510245
2252     ERR4797048
Name: wgs_id, Length: 11920, dtype: category
Categories (11920, object): ['ERR4829684' < 'ERR8975196' < 'ERR3287611' < 'ERR4828904' ... 'ERR4797355' < 'ERR4811968' < 'ERR2510245' < 'ERR4797048']

# Getting rpob aa snp data

In [ ]:
all_snp

In [84]:
variants_ind = variants[variants['gene']=='rpoB']
aa_rpoB = []
all_snp = variants_ind['SNP'].unique()

for x in tqdm(variants['sample_id'].unique()):
    if x in variants_ind['sample_id'].unique():
        aa_rpoB.append(compare_snp_lists_with_values(all_snp, variants_ind[variants_ind['sample_id']==x]['SNP'].to_list(), variants_ind[variants_ind['sample_id']==x]['freq'].to_list()))
    else:
        aa_rpoB.append([0]*len(all_snp))
aa_rpoB_array = np.array(aa_rpoB)
np.savetxt('data_aa/aa_rpoB.csv', aa_rpoB_array, delimiter=',')   # X is an array

100%|██████████| 11920/11920 [00:31<00:00, 373.60it/s]


In [131]:
np.savetxt('data_aa/train_aa_rpoB.csv', aa_rpoB_array[:int(0.9*aa_rpoB_array.shape[0])], delimiter=',')   # X 
np.savetxt('data_aa/test_aa_rpoB.csv', aa_rpoB_array[int(0.9*aa_rpoB_array.shape[0]):], delimiter=',')   # X 

In [ ]:
np.savetxt('data_aa/aa_rpoB.csv', aa_rpoB_array, delimiter=',')   # X is an array

# Getting eth aa snp data

In [87]:
variants_ind = variants[variants['gene'].isin(['embA','embB','embC'])]
aa_emb = []
all_snp = variants_ind['SNP'].unique()

for x in tqdm(variants['sample_id'].unique()):
    if x in variants_ind['sample_id'].unique():
        aa_emb.append(compare_snp_lists_with_values(all_snp, variants_ind[variants_ind['sample_id']==x]['SNP'].to_list(), variants_ind[variants_ind['sample_id']==x]['freq'].to_list()))
    else:
        aa_emb.append([0]*len(all_snp))
aa_emb_array = np.array(aa_emb)
np.savetxt('data_aa/aa_emb.csv', aa_emb_array, delimiter=',')   # X is an array

100%|██████████| 11920/11920 [02:10<00:00, 91.03it/s]


In [132]:
np.savetxt('data_aa/aa_emb.csv', aa_emb_array, delimiter=',')   # X is an array
np.savetxt('data_aa/train_aa_emb.csv', aa_emb_array[:int(0.9*aa_emb_array.shape[0])], delimiter=',')   # X 
np.savetxt('data_aa/test_aa_emb.csv', aa_emb_array[int(0.9*aa_emb_array.shape[0]):], delimiter=',')   # X 

# Getting corresponding mic snp dataset for the above

In [113]:
cryptic_mic['wgs_id'] = pd.Categorical(cryptic_mic['wgs_id'], categories=variants['sample_id'].unique().tolist(), ordered=True)
# Sort the DataFrame by the 'wgs_id' column

cryptic_mic_ordered = cryptic_mic.sort_values('wgs_id')
overlap = set(variants['sample_id']).intersection(set(cryptic['wgs_id'].to_list()))
cryptic_mic_ordered = cryptic_mic_ordered[cryptic_mic_ordered['wgs_id'].isin(overlap)]

In [114]:
np.savetxt('data_aa/ETH_MIC.csv', np.array([float(x) for x in cryptic_mic_ordered['ETH_MIC']]), delimiter=',')   # X is an array
np.savetxt('data_aa/train_ETH_MIC.csv', np.array([float(x) for x in cryptic_mic_ordered['ETH_MIC']])[:int(0.9*cryptic_mic_ordered.shape[0])], delimiter=',')   # X is an array
np.savetxt('data_aa/test_ETH_MIC.csv', np.array([float(x) for x in cryptic_mic_ordered['ETH_MIC']])[int(0.9*cryptic_mic_ordered.shape[0]):], delimiter=',')   # X is an array

np.savetxt('data_aa/RIF_MIC.csv', np.array([float(x) for x in cryptic_mic_ordered['RIF_MIC']]), delimiter=',')   # X is an array
np.savetxt('data_aa/train_RIF_MIC.csv', np.array([float(x) for x in cryptic_mic_ordered['RIF_MIC']])[:int(0.9*cryptic_mic_ordered.shape[0])], delimiter=',')   # X is an array
np.savetxt('data_aa/test_RIF_MIC.csv', np.array([float(x) for x in cryptic_mic_ordered['RIF_MIC']])[int(0.9*cryptic_mic_ordered.shape[0]):], delimiter=',')   # X is an array

In [92]:
cryptic_mic['ETH_MIC']

Index(['UNIQUEID', 'AMI_BINARY_PHENOTYPE', 'BDQ_BINARY_PHENOTYPE',
       'CFZ_BINARY_PHENOTYPE', 'DLM_BINARY_PHENOTYPE', 'EMB_BINARY_PHENOTYPE',
       'ETH_BINARY_PHENOTYPE', 'INH_BINARY_PHENOTYPE', 'KAN_BINARY_PHENOTYPE',
       'LEV_BINARY_PHENOTYPE', 'LZD_BINARY_PHENOTYPE', 'MXF_BINARY_PHENOTYPE',
       'RIF_BINARY_PHENOTYPE', 'RFB_BINARY_PHENOTYPE', 'AMI_MIC', 'BDQ_MIC',
       'CFZ_MIC', 'DLM_MIC', 'EMB_MIC', 'ETH_MIC', 'INH_MIC', 'KAN_MIC',
       'LEV_MIC', 'LZD_MIC', 'MXF_MIC', 'RIF_MIC', 'RFB_MIC',
       'AMI_PHENOTYPE_QUALITY', 'BDQ_PHENOTYPE_QUALITY',
       'CFZ_PHENOTYPE_QUALITY', 'EMB_PHENOTYPE_QUALITY',
       'ETH_PHENOTYPE_QUALITY', 'INH_PHENOTYPE_QUALITY',
       'KAN_PHENOTYPE_QUALITY', 'LEV_PHENOTYPE_QUALITY',
       'LZD_PHENOTYPE_QUALITY', 'MXF_PHENOTYPE_QUALITY',
       'RIF_PHENOTYPE_QUALITY', 'RFB_PHENOTYPE_QUALITY', 'ENA_SAMPLE', 'VCF',
       'REGENOTYPED_VCF', 'wgs_id'],
      dtype='object')

In [ ]:
from tqdm import tqdm
import numpy as np

def convert_to_coordinate_value(input_list):
    return [(i, val) for i, val in enumerate(input_list) if val != 0]

# Generate coordinate values
aa_coord = [convert_to_coordinate_value(x) for x in tqdm(aa_data_freq)]

# Find the length of the longest list
max_length = max(len(item) for item in aa_coord)

# Pad the lists to have uniform length
padded_aa_coord = [item + [(0, 0)] * (max_length - len(item)) for item in aa_coord]

# Convert to a high-dimensional NumPy array
aa_coord_array = np.array(padded_aa_coord)

# The result is a high-dimensional NumPy array
print(aa_coord_array.shape)


In [ ]:
from tqdm import tqdm
import numpy as np

def convert_to_coordinate_value(input_list):
    return [(i, val) for i, val in enumerate(input_list) if val != 0]

# Generate coordinate values
aa_coord = [convert_to_coordinate_value(x) for x in tqdm(aa_data_freq)]

# Find the length of the longest list
max_length = max(len(item) for item in aa_coord)

# Pad the lists to have uniform length
padded_aa_coord = [item + [(0, 0)] * (max_length - len(item)) for item in aa_coord]

# Convert to a high-dimensional NumPy array
aa_coord_array = np.array(padded_aa_coord)

# The result is a high-dimensional NumPy array
print(aa_coord_array.shape)


In [ ]:
for x in np.unique(variants['sample_id']):
     print(variants[variants['sample_id'] == 'ERR9787047'].shape)

In [7]:
variants['change'].value_counts()

n.-187C>T               135403
p.Glu21Gln              133939
c.33A>G                 133602
c.-165T>C               133108
p.Ile948Val             133064
                         ...  
c.-4164_*1837del             1
p.Ala876Ser                  1
c.2192_2193insGCCTAG         1
c.536_537insGCCCG            1
n.1011T>G                    1
Name: change, Length: 109920, dtype: int64

# Classification targets

In [82]:
cryptic_clean = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data1/CRyPTIC_reuse_table_20221019_cleaned.csv')

In [79]:
rif_target = pd.read_csv('data_aa/RIF_MIC.csv',header=None)

In [84]:
cryptic_clean['RIF_MIC']

0         0.12
1         0.12
2         0.03
3         0.12
4         0.03
         ...  
12280    16.00
12281    16.00
12282    16.00
12283     0.06
12284    16.00
Name: RIF_MIC, Length: 12285, dtype: float64

In [19]:
print(np.sort(np.unique(rif_target.values)))

[1.5e-02 3.0e-02 6.0e-02 1.2e-01 2.5e-01 5.0e-01 1.0e+00 2.0e+00 4.0e+00
 8.0e+00 1.6e+01     nan]


In [22]:
cryptic_mic_ordered.columns

Index(['UNIQUEID', 'AMI_BINARY_PHENOTYPE', 'BDQ_BINARY_PHENOTYPE',
       'CFZ_BINARY_PHENOTYPE', 'DLM_BINARY_PHENOTYPE', 'EMB_BINARY_PHENOTYPE',
       'ETH_BINARY_PHENOTYPE', 'INH_BINARY_PHENOTYPE', 'KAN_BINARY_PHENOTYPE',
       'LEV_BINARY_PHENOTYPE', 'LZD_BINARY_PHENOTYPE', 'MXF_BINARY_PHENOTYPE',
       'RIF_BINARY_PHENOTYPE', 'RFB_BINARY_PHENOTYPE', 'AMI_MIC', 'BDQ_MIC',
       'CFZ_MIC', 'DLM_MIC', 'EMB_MIC', 'ETH_MIC', 'INH_MIC', 'KAN_MIC',
       'LEV_MIC', 'LZD_MIC', 'MXF_MIC', 'RIF_MIC', 'RFB_MIC',
       'AMI_PHENOTYPE_QUALITY', 'BDQ_PHENOTYPE_QUALITY',
       'CFZ_PHENOTYPE_QUALITY', 'EMB_PHENOTYPE_QUALITY',
       'ETH_PHENOTYPE_QUALITY', 'INH_PHENOTYPE_QUALITY',
       'KAN_PHENOTYPE_QUALITY', 'LEV_PHENOTYPE_QUALITY',
       'LZD_PHENOTYPE_QUALITY', 'MXF_PHENOTYPE_QUALITY',
       'RIF_PHENOTYPE_QUALITY', 'RFB_PHENOTYPE_QUALITY', 'ENA_SAMPLE', 'VCF',
       'REGENOTYPED_VCF', 'wgs_id'],
      dtype='object')

In [25]:
for x in ['AMI_MIC', 'BDQ_MIC','CFZ_MIC', 'DLM_MIC', 'EMB_MIC', 'ETH_MIC', 'INH_MIC', 'KAN_MIC','LEV_MIC', 'LZD_MIC', 'MXF_MIC', 'RIF_MIC', 'RFB_MIC']:
    print(x)
    print(np.sort(np.unique(cryptic_mic_ordered[x].values)))

AMI_MIC
[ 0.125  0.5    1.     2.     4.     8.    16.    32.       nan]
BDQ_MIC
[0.004  0.0075 0.015  0.03   0.06   0.12   0.25   0.5    1.     2.
 4.        nan]
CFZ_MIC
[0.015 0.03  0.06  0.12  0.25  0.5   1.    2.    4.    8.      nan]
DLM_MIC
[0.004  0.0075 0.015  0.03   0.06   0.12   0.25   0.5    1.     2.
    nan]
EMB_MIC
[3.00e-02 1.20e-01 1.25e-01 2.50e-01 5.00e-01 1.00e+00 2.00e+00 4.00e+00
 8.00e+00 1.60e+01 3.20e+01 6.40e+01      nan]
ETH_MIC
[ 0.125  0.5    1.     2.     4.     8.    16.       nan]
INH_MIC
[1.125e-02 5.000e-02 1.000e-01 2.000e-01 4.000e-01 8.000e-01 1.600e+00
 3.200e+00 6.400e+00 1.280e+01 2.560e+01       nan]
KAN_MIC
[ 0.5  2.   4.   8.  16.  32.   nan]
LEV_MIC
[ 0.06  0.25  0.5   1.    2.    4.    8.   16.     nan]
LZD_MIC
[0.03 0.06 0.12 0.15 0.25 0.5  1.   2.   4.   8.    nan]
MXF_MIC
[0.03 0.12 0.25 0.5  1.   2.   4.   8.    nan]
RIF_MIC
[1.5e-02 3.0e-02 6.0e-02 1.2e-01 2.5e-01 5.0e-01 1.0e+00 2.0e+00 4.0e+00
 8.0e+00 1.6e+01     nan]
RFB_MIC
[0.03 0

In [90]:
import pandas as pd
import numpy as np

# Sample data
data = {
    'Values': [1.5e-02, 3.0e-02, 6.0e-02, 1.2e-01, 2.5e-01, 5.0e-01, 1.0e+00, 2.0e+00, 4.0e+00, 8.0e+00, 1.6e+01, np.nan]
}

def one_hot_encode_df(data):
    # Create a DataFrame from the data
    df = pd.DataFrame(data)
    
    # One-hot encode the data, converting NaN values as well
    # 'prefix' can be adjusted based on preference for column naming
    one_hot_encoded_df = pd.get_dummies(df.iloc[:,0], dummy_na=True)
    
    return one_hot_encoded_df

df = one_hot_encode_df(data)
print(df)


    0.015   0.030   0.060   0.120   0.250   0.500   1.000   2.000   4.000   \
0        1       0       0       0       0       0       0       0       0   
1        0       1       0       0       0       0       0       0       0   
2        0       0       1       0       0       0       0       0       0   
3        0       0       0       1       0       0       0       0       0   
4        0       0       0       0       1       0       0       0       0   
5        0       0       0       0       0       1       0       0       0   
6        0       0       0       0       0       0       1       0       0   
7        0       0       0       0       0       0       0       1       0   
8        0       0       0       0       0       0       0       0       1   
9        0       0       0       0       0       0       0       0       0   
10       0       0       0       0       0       0       0       0       0   
11       0       0       0       0       0       0       0      

In [91]:
cryptic_clean['RIF_MIC'].unique()

array([1.2e-01, 3.0e-02, 8.0e+00, 2.0e+00,     nan, 2.5e-01, 1.0e+00,
       5.0e-01, 4.0e+00, 1.6e+01, 1.5e-02, 6.0e-02])

In [110]:
cryptic_clean

,UNIQUEID,AMI_BINARY_PHENOTYPE,BDQ_BINARY_PHENOTYPE,CFZ_BINARY_PHENOTYPE,DLM_BINARY_PHENOTYPE,EMB_BINARY_PHENOTYPE,ETH_BINARY_PHENOTYPE,INH_BINARY_PHENOTYPE,KAN_BINARY_PHENOTYPE,LEV_BINARY_PHENOTYPE,...,KAN_PHENOTYPE_QUALITY,LEV_PHENOTYPE_QUALITY,LZD_PHENOTYPE_QUALITY,MXF_PHENOTYPE_QUALITY,RIF_PHENOTYPE_QUALITY,RFB_PHENOTYPE_QUALITY,ENA_SAMPLE,VCF,REGENOTYPED_VCF,wgs_id
0,site.05.subj.PTAN-0001.lab.TAN-001.iso.1,S,S,S,S,S,S,S,S,S,...,HIGH,LOW,HIGH,HIGH,HIGH,HIGH,ERS1987021,00/00/04/90/490/site.05.iso.1.subject.PTAN-000...,00/00/04/90/490/site.05.iso.1.subject.PTAN-000...,ERR2184202
1,site.05.subj.PTAN-0002.lab.TAN-004.iso.1,S,S,S,S,S,S,S,S,S,...,HIGH,HIGH,LOW,LOW,LOW,HIGH,ERS1987022,00/00/04/91/491/site.05.iso.1.subject.PTAN-000...,00/00/04/91/491/site.05.iso.1.subject.PTAN-000...,ERR2184203
2,site.05.subj.PTAN-0003.lab.TAN-006.iso.1,S,S,S,S,S,S,S,S,S,...,HIGH,HIGH,HIGH,LOW,HIGH,HIGH,ERS1987023,00/00/04/92/492/site.05.iso.1.subject.PTAN-000...,00/00/04/92/492/site.05.iso.1.subject.PTAN-000...,ERR2184204
3,site.05.subj.PTAN-0004.lab.TAN-007.iso.1,S,S,S,S,S,NaN,S,S,S,...,HIGH,HIGH,LOW,HIGH,HIGH,HIGH,ERS1987024,00/00/04/93/493/site.05.iso.1.subject.PTAN-000...,00/00/04/93/493/site.05.iso.1.subject.PTAN-000...,ERR2184205
4,site.05.subj.PTAN-0005.lab.TAN-010.iso.1,S,S,S,S,S,S,S,S,S,...,HIGH,HIGH,LOW,HIGH,HIGH,HIGH,ERS1987025,00/00/04/94/494/site.05.iso.1.subject.PTAN-000...,00/00/04/94/494/site.05.iso.1.subject.PTAN-000...,ERR2184206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12280,site.11.subj.XTB_18-071.lab.XTB_18-071.iso.1,S,S,S,S,R,R,R,S,R,...,MEDIUM,MEDIUM,MEDIUM,MEDIUM,MEDIUM,MEDIUM,ERS6421155,00/01/47/01/14701/site.11.iso.1.subject.XTB_18...,00/01/47/01/14701/site.11.iso.1.subject.XTB_18...,SAMEA8741572
12281,site.11.subj.XTB_18-072.lab.XTB_18-072.iso.1,S,S,S,S,I,I,R,S,R,...,MEDIUM,MEDIUM,MEDIUM,MEDIUM,MEDIUM,MEDIUM,ERS6421156,00/01/47/02/14702/site.11.iso.1.subject.XTB_18...,00/01/47/02/14702/site.11.iso.1.subject.XTB_18...,SAMEA8741573
12282,site.11.subj.XTB_18-074.lab.XTB_18-074.iso.1,S,S,S,S,I,R,R,R,S,...,MEDIUM,MEDIUM,MEDIUM,MEDIUM,MEDIUM,MEDIUM,ERS6421157,00/01/47/03/14703/site.11.iso.1.subject.XTB_18...,00/01/47/03/14703/site.11.iso.1.subject.XTB_18...,SAMEA8741574
12283,site.11.subj.XTB_18-075.lab.XTB_18-075.iso.1,S,S,S,S,S,R,R,S,S,...,MEDIUM,MEDIUM,MEDIUM,MEDIUM,MEDIUM,MEDIUM,ERS6421158,00/01/47/04/14704/site.11.iso.1.subject.XTB_18...,00/01/47/04/14704/site.11.iso.1.subject.XTB_18...,SAMEA8741575


In [94]:
df.columns

Float64Index([0.015, 0.03, 0.06, 0.12, 0.25, 0.5, 1.0, 2.0, 4.0, 8.0, 16.0,
              nan],
             dtype='float64')

In [104]:
df = one_hot_encode_df(cryptic_clean['RIF_MIC'])
df.drop(columns=np.nan, inplace=True)

In [105]:
df

,0.015,0.030,0.060,0.120,0.250,0.500,1.000,2.000,4.000,8.000,16.000
0,0,0,0,1,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
12280,0,0,0,0,0,0,0,0,0,0,1
12281,0,0,0,0,0,0,0,0,0,0,1
12282,0,0,0,0,0,0,0,0,0,0,1
12283,0,0,1,0,0,0,0,0,0,0,0


In [106]:
np.array([float(x) for x in cryptic_mic_ordered['RIF_MIC']])[:int(0.9*cryptic_mic_ordered.shape[0])]

array([0.12, 0.03, 0.25, ..., 0.25, 8.  , 0.06])

In [107]:
# np.savetxt('data_aa/RIF_MIC.csv', np.array([float(x) for x in cryptic_mic_ordered['RIF_MIC']]), delimiter=',')   # X is an array
# np.savetxt('data_aa/train_RIF_MIC.csv', np.array([float(x) for x in cryptic_mic_ordered['RIF_MIC']])[:int(0.9*cryptic_mic_ordered.shape[0])], delimiter=',')   # X is an array
# np.savetxt('data_aa/test_RIF_MIC.csv', np.array([float(x) for x in cryptic_mic_ordered['RIF_MIC']])[int(0.9*cryptic_mic_ordered.shape[0]):], delimiter=',')   # X is an array

In [108]:
df.to_csv('data_aa/RIF_MIC_class.csv', index=False, header=True)

In [109]:
df.iloc[:int(0.9*cryptic_mic_ordered.shape[0]), :].to_csv('data_aa/train_RIF_MIC_class.csv', index=False, header=True)
df.iloc[int(0.9*cryptic_mic_ordered.shape[0]):, :].to_csv('data_aa/test_RIF_MIC_class.csv', index=False, header=True)

# fm Data

In [91]:
fm_aa = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data_aa/cryptic_fm_transposed.csv')
fm_aa = fm_aa.loc[:, fm_aa.columns!='NA']
fm_aa = fm_aa.transpose()
fm_aa = fm_aa.iloc[1:,:]

In [113]:
fm_aa.to_csv('data_aa/fm_aa.csv', index=False)

In [108]:
mic_aa = cryptic_mic[cryptic_mic['wgs_id'].isin(fm_aa.index.unique())]#.iloc[:,14:27]
mic_aa['wgs_id'] = pd.Categorical(mic_aa['wgs_id'], categories=fm_aa.index.unique().tolist(), ordered=True)
mic_aa = mic_aa.sort_values('wgs_id')

/tmp/ipykernel_106569/1595918224.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mic_aa['wgs_id'] = pd.Categorical(mic_aa['wgs_id'], categories=fm_aa.index.unique().tolist(), ordered=True)


In [112]:
mic_aa = mic_aa.iloc[:,14:27].astype('float')
mic_aa.to_csv('data_aa/mic_aa_fm.csv', index=False)

In [114]:
mic_aa.iloc[:int(mic_aa.shape[0]*0.9),:].to_csv('data_aa/mic_aa_fm_train.csv', index=False)
mic_aa.iloc[int(mic_aa.shape[0]*0.9):,:].to_csv('data_aa/mix_aa_fm_test.csv', index=False)

fm_aa.iloc[:int(fm_aa.shape[0]*0.9),:].to_csv('data_aa/fm_aa_train.csv', index=False)
fm_aa.iloc[int(fm_aa.shape[0]*0.9):,:].to_csv('data_aa/fm_aa_test.csv', index=False)

In [104]:
fm_aa.index.unique()

Index(['ERR4810491', 'ERR4810493', 'ERR4810494', 'ERR4810495', 'ERR4810497',
       'ERR4810498', 'ERR4810507', 'ERR4810509', 'ERR4810512', 'ERR4810516',
       ...
       'ERR8699755', 'ERR8686563', 'ERR8699762', 'ERR8699756', 'ERR8699805',
       'ERR8699231', 'ERR8699764', 'ERR8699789', 'ERR8699232', 'ERR8699267'],
      dtype='object', length=10824)

In [84]:
Nina_aa.transpose()

,0,1,2,3,4,5,6,7,8,9,...,9616,9617,9618,9619,9620,9621,9622,9623,9624,9625
Unnamed: 0,Chromosome_11_C,Chromosome_22_C,Chromosome_29_G,Chromosome_31_G,Chromosome_69_T,Chromosome_71_T,Chromosome_76_C,Chromosome_78_G,Chromosome_82_C,Chromosome_88_A,...,Chromosome_4338506_A,Chromosome_4338508_C,Chromosome_4338508_G,Chromosome_4338513_A,Chromosome_4338514_C,Chromosome_4338514_G,Chromosome_4338516_TCGCATTAACCGACGGGTCGGCCCAGG...,Chromosome_4338518_G,Chromosome_4338520_G,Chromosome_4338521_A
ERR4810489,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ERR4810491,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ERR4810493,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ERR4810494,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ERR8699764,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ERR8699789,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ERR8699232,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ERR8699267,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
